In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Load the dataset
data = pd.read_csv('../3_cluster_fight_style/data_with_win_percentages.csv')

# Inspect the data
display(data.head())

,end_time,f1,f2,fight_date,method,referee,round,time_format,weightclass,winner,...,f2_td_def_,f2_total_str,f2_weight,gender,year,name_x,winner_cluster,name_y,loser_cluster,win_percentage
0,5:00,Lerone Murphy,Edson Barboza,2024-05-18,Decision - Unanimous,Herb Dean,5,5,featherweight,f1,...,72%,79 of 242,65.77,M,2024,Lerone Murphy,4,Edson Barboza,4,50.0
1,5:00,Themba Gorimbo,Ramiz Brahimaj,2024-05-18,Decision - Unanimous,Mark Smith,3,3,welterweight,f1,...,44%,15 of 20,77.11,M,2024,Themba Gorimbo,2,Ramiz Brahimaj,2,50.0
2,5:00,Abus Magomedov,Warlley Alves,2024-05-18,Decision - Unanimous,Dan Miragliotta,3,3,middleweight,f1,...,62%,17 of 39,83.91,M,2024,Abus Magomedov,2,Warlley Alves,2,50.0
3,5:00,Alatengheili,Kleydson Rodrigues,2024-05-18,Decision - Unanimous,Herb Dean,3,3,bantamweight,f1,...,53%,70 of 126,61.23,M,2024,Alatengheili,2,Kleydson Rodrigues,4,52.0
4,5:00,Vanessa Demopoulos,Emily Ducote,2024-05-18,Decision - Split,Chris Tognoni,3,3,strawweight,f1,...,94%,147 of 218,52.16,M,2024,Vanessa Demopoulos,1,Emily Ducote,4,25.0


In [9]:
# Feature engineering
data['model_f1_win'] = (data['winner_cluster'] == 'f1').astype(int)  # Assuming 'f1' and 'f2' are in 'winner_cluster'

# Select relevant features for prediction
features = ['winner_cluster', 'loser_cluster', 'win_percentage']  # Add other relevant features as needed
X = pd.get_dummies(data[features], drop_first=True)  # Convert categorical variables to dummy/indicator variables
y = data['model_f1_win']

In [5]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [6]:
# Train a RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [7]:
# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 1.0
Confusion Matrix:
 [[1024]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      1024

    accuracy                           1.00      1024
   macro avg       1.00      1.00      1.00      1024
weighted avg       1.00      1.00      1.00      1024

